In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle("../data/train_IOB_repl_compound.pkl")

In [17]:
new_train_df = pd.DataFrame()
for _, g in df.groupby('_id'):
    if (g.cat_raw_material_headline == 1).any():
        new_train_df = new_train_df.append(g.loc[g.cat_raw_material_headline == 1])
    else:
        new_train_df = new_train_df.append(g)

In [19]:
new_train_df.raw_material_tag_seq.apply(lambda x: 'B' in x).sum()

727

In [20]:
df.raw_material_tag_seq.apply(lambda x: 'B' in x).sum()

1106